In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pytz
from pandas.tseries.offsets import DateOffset

# --- CONFIGURACIÓN ---
TOKEN = "7c7245244cd2df18b2b03e0834258450b2ab7c578910115fb8975a7f1c48b9e8"
BASE_URL = "https://www.banxico.org.mx/SieAPIRest/service/v1/"
INDICADOR_INFLACION = "SP1"

# --- DICCIONARIO PARA FORZAR ESPAÑOL ---
MESES_ES = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

def obtener_datos_banxico(clave_indicador, fecha_inicio, fecha_fin):
    url = f"{BASE_URL}series/{clave_indicador}/datos/{fecha_inicio}/{fecha_fin}"
    headers = {"Bmx-Token": TOKEN}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        datos = response.json()
        serie = datos['bmx']['series'][0]['datos']
        df = pd.DataFrame(serie)
        df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
        df['dato'] = pd.to_numeric(df['dato'], errors='coerce')
        return df
    except (requests.RequestException, KeyError, IndexError, ValueError):
        return pd.DataFrame()

def intentar_calculo_inflacion(mes_objetivo):
    mes_comparativo = mes_objetivo - DateOffset(years=1)
    f_inicio = (mes_comparativo - DateOffset(days=5)).strftime('%Y-%m-%d')
    f_fin = (mes_objetivo + DateOffset(days=5)).strftime('%Y-%m-%d')

    df = obtener_datos_banxico(INDICADOR_INFLACION, f_inicio, f_fin)

    if df.empty:
        return None, None

    def get_dato_mes(df, fecha_target):
        filtro = df[
            (df['fecha'].dt.year == fecha_target.year) &
            (df['fecha'].dt.month == fecha_target.month)
        ]
        return filtro.sort_values('fecha').iloc[-1]['dato'] if not filtro.empty else None

    valor_actual = get_dato_mes(df, mes_objetivo)
    valor_anterior = get_dato_mes(df, mes_comparativo)

    if valor_actual is not None and valor_anterior is not None:
        inflacion = ((valor_actual - valor_anterior) / valor_anterior) * 100

        # --- CORRECCIÓN AQUÍ: Construcción manual de la fecha en español ---
        nombre_mes_es = MESES_ES[mes_objetivo.month]
        texto_fecha = f"{nombre_mes_es} {mes_objetivo.year}"

        return inflacion, texto_fecha

    return None, None

# --- EJECUCIÓN PRINCIPAL ---
zona_mx = pytz.timezone('America/Mexico_City')
hoy = datetime.now(zona_mx)
meses_atras_a_probar = [1, 2]
resultado_encontrado = False

for i in meses_atras_a_probar:
    # Solo para el log usamos el diccionario también si queremos, o strftime estándar
    mes_referencia = datetime(hoy.year, hoy.month, 1) - DateOffset(months=i)
    nombre_mes_log = f"{MESES_ES[mes_referencia.month]} {mes_referencia.year}"

    inflacion, nombre_fecha_final = intentar_calculo_inflacion(mes_referencia)

    if inflacion is not None:
        # Aquí la salida ya será 100% en español gracias a la función
        print(f"-> Inflación anual estimada para {nombre_fecha_final}: {inflacion:.2f}%")
        resultado_encontrado = True
        break
    else:
        print(f"⚠️ Aún no hay datos para {nombre_mes_log}.")
        if i == 1:
            print("*  Intentando con el mes anterior...")

if not resultado_encontrado:
    print("❌ ERROR CRÍTICO: No hay datos recientes disponibles.")

Inflación anual estimada para Octubre 2025: 3.57%
